# Scratchpad for testing out graphlab with project data

In [1]:
import numpy as np
import graphlab
import pandas as pd
import matplotlib.pyplot as plt

[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1467047207.log


This non-commercial license of GraphLab Create is assigned to windsurf_dean@yahoo.com and will expire on May 26, 2017. For commercial licensing options, visit https://dato.com/buy/.


In [2]:
# create Pandas DF with data pulled from Postgres
df = pd.read_csv('data_analysis/user_reviews.csv', header=None,
                 names=['user_id', 'business_id', 'bus_name', 'stars', 'locale'])

In [28]:
df.head()

,user_id,business_id,bus_name,stars,locale
0,Ouus3lC3Xk3YRGx4ptd_4A,DW6SI2KxcfXBHeo9jEr4lQ,Pyramid Cafe,2,"Las Vegas, NV"
1,dz_sBsL69aWzsxksT2O_Mg,DW6SI2KxcfXBHeo9jEr4lQ,Pyramid Cafe,2,"Las Vegas, NV"
2,zyI-u0C1YOzp5v1j2wuCOQ,DW6SI2KxcfXBHeo9jEr4lQ,Pyramid Cafe,3,"Las Vegas, NV"
3,vyfsQo-estP8EfiIFMsL6g,DW6SI2KxcfXBHeo9jEr4lQ,Pyramid Cafe,3,"Las Vegas, NV"
4,lxZSVeJz6KEBW1nlA3JKJg,DW6SI2KxcfXBHeo9jEr4lQ,Pyramid Cafe,1,"Las Vegas, NV"


In [3]:
# store data in Graphlab's SFrame type
sf = graphlab.SFrame(df[['user_id', 'business_id', 'stars']])

In [29]:
sf.head()

user_id,business_id,stars
Ouus3lC3Xk3YRGx4ptd_4A,DW6SI2KxcfXBHeo9jEr4lQ,2
dz_sBsL69aWzsxksT2O_Mg,DW6SI2KxcfXBHeo9jEr4lQ,2
zyI-u0C1YOzp5v1j2wuCOQ,DW6SI2KxcfXBHeo9jEr4lQ,3
vyfsQo-estP8EfiIFMsL6g,DW6SI2KxcfXBHeo9jEr4lQ,3
lxZSVeJz6KEBW1nlA3JKJg,DW6SI2KxcfXBHeo9jEr4lQ,1
CbMCOeeHVeafcD-6-CJrPg,DW6SI2KxcfXBHeo9jEr4lQ,4
Z8DyUMojRhFbLyHBymM_rA,DW6SI2KxcfXBHeo9jEr4lQ,2
sXWvDlMQspjJ1zzr3cEM7Q,DW6SI2KxcfXBHeo9jEr4lQ,2
G8RaaiVzbyRvsBmoUx0VeA,DW6SI2KxcfXBHeo9jEr4lQ,1
f0isFKnJVZVmpdCPNd8Gfw,DW6SI2KxcfXBHeo9jEr4lQ,1


In [16]:
sf.column_names

<bound method SFrame.column_names of Columns:
	user_id	str
	business_id	str
	stars	int

Rows: 1363242

Data:
+------------------------+------------------------+-------+
|        user_id         |      business_id       | stars |
+------------------------+------------------------+-------+
| Ouus3lC3Xk3YRGx4ptd_4A | DW6SI2KxcfXBHeo9jEr4lQ |   2   |
| dz_sBsL69aWzsxksT2O_Mg | DW6SI2KxcfXBHeo9jEr4lQ |   2   |
| zyI-u0C1YOzp5v1j2wuCOQ | DW6SI2KxcfXBHeo9jEr4lQ |   3   |
| vyfsQo-estP8EfiIFMsL6g | DW6SI2KxcfXBHeo9jEr4lQ |   3   |
| lxZSVeJz6KEBW1nlA3JKJg | DW6SI2KxcfXBHeo9jEr4lQ |   1   |
| CbMCOeeHVeafcD-6-CJrPg | DW6SI2KxcfXBHeo9jEr4lQ |   4   |
| Z8DyUMojRhFbLyHBymM_rA | DW6SI2KxcfXBHeo9jEr4lQ |   2   |
| sXWvDlMQspjJ1zzr3cEM7Q | DW6SI2KxcfXBHeo9jEr4lQ |   2   |
| G8RaaiVzbyRvsBmoUx0VeA | DW6SI2KxcfXBHeo9jEr4lQ |   1   |
| f0isFKnJVZVmpdCPNd8Gfw | DW6SI2KxcfXBHeo9jEr4lQ |   1   |
+------------------------+------------------------+-------+
[1363242 rows x 3 columns]
Note: Only the head of t

In [5]:
# create the recommender (will train during this step)
rec = graphlab.recommender.factorization_recommender.create(
            sf,
            user_id='user_id',
            item_id='business_id',
            target='stars',
            solver='als',
            side_data_factorization=False)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 1363242 observations with 388612 users and 24974 items.

Data prepared in: 2.20518s

Training factorization_recommender for recommendations.

...

| 48      | 1m 0s        | 0.426188               |

| 49      | 1m 2s        | 0.42563                |

| FINAL   | 1m 2s        | 0.42563                |

+---------+--------------+------------------------+

Optimization Complete: Iteration limit reached.

In [30]:
rec = graphlab.recommender.create(
            sf,
            user_id='user_id',
            item_id='business_id',
            target ='stars')

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 1363242 observations with 388612 users and 24974 items.

Data prepared in: 2.91034s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.397275

Final training RMSE: 0.356335

In [31]:
# evaluate 1 datapoint (user=1, business=100) for potential rating
one_datapoint_sf = graphlab.SFrame({'user_id': [1], 'business_id': [100]})
print "stars:", rec.predict(one_datapoint_sf)[0]   # 4.879

stars: 3.73693885605


In [32]:
rec.list_fields()

['adagrad_momentum_weighting',
 'additional_iterations_if_unhealthy',
 'binary_target',
 'coefficients',
 'data_load_time',
 'ials_confidence_scaling_factor',
 'ials_confidence_scaling_type',
 'init_random_sigma',
 'item_id',
 'item_side_data_column_names',
 'item_side_data_column_types',
 'linear_regularization',
 'max_iterations',
 'model_name',
 'nmf',
 'num_factors',
 'num_features',
 'num_item_side_features',
 'num_items',
 'num_observations',
 'num_sampled_negative_examples',
 'num_tempering_iterations',
 'num_user_side_features',
 'num_users',
 'observation_data_column_names',
 'random_seed',
 'ranking_regularization',
 'regularization',
 'regularization_type',
 'sgd_convergence_interval',
 'sgd_convergence_threshold',
 'sgd_max_trial_iterations',
 'sgd_sampling_block_size',
 'sgd_step_adjustment_interval',
 'sgd_step_size',
 'sgd_trial_sample_minimum_size',
 'sgd_trial_sample_proportion',
 'side_data_factorization',
 'solver',
 'step_size_decrease_rate',
 'target',
 'tempering_

In [33]:
rec.get('coefficients')['intercept']

3.73693885605052

In [34]:
# rec['coefficients']  # equivalent to: rec.get('coefficients')
rec.get('coefficients')['user_id']

user_id,linear_terms,factors
Ouus3lC3Xk3YRGx4ptd_4A,-0.15065652132,"[0.272025257349,0.0420655496418, ..."
dz_sBsL69aWzsxksT2O_Mg,-0.108334258199,"[0.1754091084,0.072174847126, ..."
zyI-u0C1YOzp5v1j2wuCOQ,-0.17282256484,"[0.0852816551924,0.0630154460669, ..."
vyfsQo-estP8EfiIFMsL6g,-0.0180961787701,"[-0.044523768127,0.258631527424, ..."
lxZSVeJz6KEBW1nlA3JKJg,-0.733900427818,"[0.282611310482,-0.185994073749, ..."
CbMCOeeHVeafcD-6-CJrPg,0.104555010796,"[0.0308729708195,-0.0298390705138, ..."
Z8DyUMojRhFbLyHBymM_rA,-0.68707960844,"[0.412124305964,-0.0944558382034, ..."
sXWvDlMQspjJ1zzr3cEM7Q,-0.62367862463,"[-0.185755431652,0.392349928617, ..."
G8RaaiVzbyRvsBmoUx0VeA,-0.690084278584,"[0.381505399942,0.330410510302, ..."
f0isFKnJVZVmpdCPNd8Gfw,-0.92436760664,"[0.37686342001,0.268129527569, ..."


In [35]:
# rec['coefficients']  # equivalent to: rec.get('coefficients')
rec.get('coefficients')['business_id']

business_id,linear_terms,factors
DW6SI2KxcfXBHeo9jEr4lQ,-1.42583227158,"[-0.106687664986,0.0529021769762, ..."
MUPWRBidG0_EQC9ey4-xKQ,-2.44162487984,"[-0.773338973522,-0.221781879663, ..."
rXEGCpdboVwczMIDZuXXaA,-1.93411910534,"[-0.0460895709693,-0.107171773911, ..."
C_lTRsrL5LQAyd3_fnoBdg,-2.22860455513,"[0.224688187242,0.589515149593, ..."
ZWPVBkb2CAQkQIn2OXTzeQ,-2.46623921394,"[-0.305990934372,-0.209893167019, ..."
SbS0XQG39VgP6L6wbZ56RA,-1.98389303684,"[-0.330567449331,-0.607387185097, ..."
eOug4Lp-b5-jrtkoLMQq2g,-2.47307372093,"[0.00786147080362,0.128409355879, ..."
FmsSGHdahYI6VarnoQhGcA,-2.35507702827,"[0.840227067471,0.174210965633, ..."
zWmMse__At85rO_ivLcpXA,-2.67476224899,"[-0.183474779129,0.323694944382, ..."
oOqsIaRvdV4La45-JJhfeQ,-2.47095966339,"[-0.138113334775,0.0903669297695, ..."


In [36]:
business_sf = rec['coefficients']['business_id']
print len(business_sf)                     # 1682
print len(business_sf['factors'][0])       # 8
user_sf = rec['coefficients']['user_id']
print len(user_sf)                      # 943
print len(user_sf['factors'][0])        # 8

24974
32
388612
32


In [37]:
import os

In [ ]:
os.path.getsize()

In [38]:
import progressbar

In [39]:
progressbar.__version__

'3.6.2'

In [ ]:
progressbar.ProgressBar.finish